In [ ]:
import nd2
import numpy as np
import pandas as pd
import h5py
import timeit
import time
import math
import statistics
import glob, os
from matplotlib import pyplot as plt

def Avg(lst): 
    return sum(lst) / len(lst) 

def makearrays(name):
    nd2name=name+".nd2"
    cell_h5=name+"diccells.h5"
    nuclear_h5=name+"nuclei.h5"
    
    my_nd2_array = nd2.imread(nd2name)   
    
    my_h5_file = h5py.File(cell_h5, 'r')
    my_h5_dset = my_h5_file['FOV0']['T0']
    my_cell_array= np.array(my_h5_dset) 
    
    my_h5_file = h5py.File(nuclear_h5, 'r')
    my_h5_dset = my_h5_file['t0']['channel0']
    my_nuclear_array= np.array(my_h5_dset) 

    return(my_nd2_array,my_cell_array,my_nuclear_array)


def nucleussizefilter(nucleararray,minsize,maxsize):
    nucleusIDlist=[]
    nucleuslist=[]
    nucleusvolumelist=[]
    #droppednucleilist

    for (x,y), value in np.ndenumerate(nucleararray):
        if value !=0:
            if value in nucleusIDlist:
                nucleuslist[nucleusIDlist.index(value)].append([x,y])
                #grab coordinates, place into list inside np array
                pass
            else:
                nucleusIDlist.append(value)
                nucleuslist.append([[x,y]])
                #grab coordinates
                
    filterednucleararray=nucleararray
    for i in range(0, len(nucleusIDlist)):
        if len(nucleuslist[i])<minsize or  len(nucleuslist[i])>maxsize:
            for j in range(0, len(nucleuslist[i])):
                filterednucleararray[nucleuslist[i][j][0],nucleuslist[i][j][1]]=0  
    return(filterednucleararray)
            


def makecelllist(my_nd2_array,my_cytoplasmic_array,my_nuclear_array):
    backgroundpixels=[]
    cellIDlist=[]
    celllist=[]

    cytoplasmicgreenlist=[]
    nucleargreenlist=[]
        
    filterednucleararray=nucleussizefilter(my_nuclear_array,125,375)
    
    for (x,y), value in np.ndenumerate(my_cytoplasmic_array):
        nuclearpixel=0
        if filterednucleararray[x,y] !=0:
            nuclearpixel=1

        red=my_nd2_array[0,x,y]
        green=my_nd2_array[1,x,y]
        if value==0:
            backgroundpixels.append([red,green])
        else:
            if nuclearpixel==0:
                cytoplasmicgreenlist.append(value)
            else:
                nucleargreenlist.append(value)


            if value in cellIDlist:
                celllist[cellIDlist.index(value)].append([red,green,x,y,nuclearpixel])
                #grab red and green, place into list inside np array
                pass
            else:
                cellIDlist.append(value)
                celllist.append([[red,green,x,y,nuclearpixel]])
                #grab red and green 
    return(backgroundpixels,cellIDlist,celllist)


def calculatebackground(backgroundpixels):
    backgroundred=[]
    backgroundgreen=[]

    for i in backgroundpixels:
        backgroundred.append(int(i[0]))
        backgroundgreen.append(int(i[1]))

    backgroundgreenmedian=statistics.median(backgroundgreen)
    backgroundredmedian=statistics.median(backgroundred)
    return(backgroundgreenmedian,backgroundredmedian)
    
    
def weightednuclearfraction(cellrank,cytoplasmnorm,nuclearnorm,celllist,background):
    normalizedgreennuclear=[]
    normalizedgreencytoplasmic=[]
    normalizedgreentotal=[]

    for i in celllist[cellrank]:
        
        if i[4]==1:
            normfactor=nuclearnorm+background
            normalizedgreennuclear.append(i[1]-normfactor)
            normalizedgreentotal.append(i[1]-normfactor)
        else:
            normfactor=cytoplasmnorm+background
            normalizedgreencytoplasmic.append(i[1]-normfactor)
            normalizedgreentotal.append(i[1]-normfactor)

    normalizedtotalgreenavg=0
    nuclearfraction=0
    dropcell=0
    
    if  len(normalizedgreencytoplasmic) !=0 and len(normalizedgreennuclear) !=0:
        nucleargreenavg= Avg(normalizedgreennuclear)
        cytoplasmicgreenavg= Avg(normalizedgreencytoplasmic)
        nuclearfraction=nucleargreenavg/cytoplasmicgreenavg
        normalizedtotalgreenavg=Avg(normalizedgreentotal)
    else:
 
        dropcell=1
    return [nuclearfraction,dropcell,cellrank,normalizedgreennuclear,normalizedgreencytoplasmic,normalizedtotalgreenavg]

def weightednuclearfractionlist(cytoplasmnorm,nuclearnorm,celllist,background):
    WeightedNuclearFractionList=[]
    AvgTotalGreenList=[]
    filteredcytoplasmicgreenlist=[]
    filterednucleargreenlist=[]
    filteredCellList=[]
    for i in range(0, len(celllist)):
    
        tempnuclearfractionout=weightednuclearfraction(i,cytoplasmnorm,nuclearnorm,celllist,background)
        dropcell=tempnuclearfractionout[1]
        tempnuclearfraction=tempnuclearfractionout[0]
    
        if dropcell==1 or tempnuclearfraction<0 :
            pass
        else:
            WeightedNuclearFractionList.append(tempnuclearfraction)

            AvgTotalGreenList.append(tempnuclearfractionout[5])
            filterednucleargreenlist.extend(tempnuclearfractionout[3])
            filteredcytoplasmicgreenlist.extend(tempnuclearfractionout[4])

            filteredCellList.append(celllist[i])

    return(np.array(WeightedNuclearFractionList),np.array(AvgTotalGreenList),filterednucleargreenlist,filteredcytoplasmicgreenlist,filteredCellList)

def pearsonsColocalization(cellrank,celllist):
    
    red=[]
    green=[]
    normalizedred=[]
    normalizedgreen=[]
    
    normredXgreen=[]
    normredsq=[]
    normgreensq=[]
    
    for i in celllist[cellrank]:
        red.append(i[0])
        green.append(i[1])
        
    redavg=(Avg(red))
    greenavg=(Avg(green))
    
    
    for i in celllist[cellrank]:
        normalizedred.append(i[0]-redavg)
        normalizedgreen.append(i[1]-greenavg)
    
    for i in range(0,len(normalizedred)):
        normredXgreen.append(normalizedred[i]*normalizedgreen[i])
        normredsq.append(pow(normalizedred[i],2))
        normgreensq.append(pow(normalizedgreen[i],2))
        
    pearsonsColocalization=sum(normredXgreen)/(pow(sum(normredsq)*sum(normgreensq),0.5))
    return pearsonsColocalization

def pearsonsColocalizationlist(celllist):#feed in filtered list
    ColocalizationList=[]
    for i in range(0, len(celllist)):
        temppearsonsColocalization=pearsonsColocalization(i,celllist)
        if math.isnan(temppearsonsColocalization) or temppearsonsColocalization==1:
            pass
        else:
            ColocalizationList.append(temppearsonsColocalization)
    return np.array(ColocalizationList)#add to image analysis



def autofluorescence(name):
    arrays=makearrays(name) 
    
    my_nd2_array=arrays[0]
    my_cytoplasmic_array=arrays[1]
    my_nuclear_array=arrays[2]
    
    makecelllistoutput=makecelllist(my_nd2_array,my_cytoplasmic_array,my_nuclear_array) 
    backgroundpixels=makecelllistoutput[0]
    celllist=makecelllistoutput[2]
    
    calculatebackgroundoutput=calculatebackground(backgroundpixels)
    backgroundgreen=calculatebackgroundoutput[0]
    
    weightednuclearfractionlistoutput=weightednuclearfractionlist(0,0,celllist,backgroundgreen)
    nucleargreenautofluorescence=Avg(weightednuclearfractionlistoutput[2])
    cytoplasmicgreenautofluorescence=Avg(weightednuclearfractionlistoutput[3])

    return (nucleargreenautofluorescence,cytoplasmicgreenautofluorescence,backgroundgreen)


def analyzeimage(name,cytoplasmnorm,nuclearnorm,pearsonscondition):
    arrays=makearrays(name) 
    
    my_nd2_array=arrays[0]
    my_cytoplasmic_array=arrays[1]
    my_nuclear_array=arrays[2]
    
    makecelllistoutput=makecelllist(my_nd2_array,my_cytoplasmic_array,my_nuclear_array) 
    backgroundpixels=makecelllistoutput[0]
    celllist=makecelllistoutput[2]
    
    calculatebackgroundoutput=calculatebackground(backgroundpixels)
    backgroundgreen=calculatebackgroundoutput[0]

    
    weightednuclearfractionlistoutput=weightednuclearfractionlist(cytoplasmnorm,nuclearnorm,celllist,backgroundgreen)
    weightednuclearfractionlisttemp=weightednuclearfractionlistoutput[0]
    totalgreenlist=weightednuclearfractionlistoutput[1]
    
    pearsonscolocalizationlisttemp= np.array([])
    filteredcelllist=weightednuclearfractionlistoutput[4]

    if pearsonscondition==1:
        pearsonscolocalizationlisttemp=pearsonsColocalizationlist(filteredcelllist)
    
    return([weightednuclearfractionlistoutput[0],weightednuclearfractionlistoutput[1],pearsonscolocalizationlisttemp],backgroundgreen)#add[1] for totalgreenavg


def plot_loghist(x):
    #plt.xlim(0.05,19.95 )
    plt.xlim(0.01,100 )

    logbins = np.logspace(-1,1,19)
    plt.hist(x, bins=logbins)
    plt.xscale('log')
    plt.title('3610 STB3 Gln condition 30 min')
    plt.xlabel('Nuclear/Cytoplasmic Marker intensity')
    plt.ylabel('Cell Count')

def plot_linhist(x):
    plt.xlim(0,1000 )
    plt.hist(x)
    plt.title('3610 STB3 Gln condition 30 min')
    plt.xlabel('Nuclear/Cytoplasmic Marker intensity')
    plt.ylabel('Cell Count')
    
def searchfornd2andmasks(folderpath, nd2list):
    nd2andmasksfound=0
    filterednd2list=[]
    
    for filename in nd2list:
        cellmask=folderpath+"\\"+filename+"diccells.h5"
        nucleusmask=folderpath+"\\"+filename+"nuclei.h5"

        if os.path.exists(cellmask) and os.path.exists(nucleusmask):
            filterednd2list.append(filename)
            nd2andmasksfound=1
                
    return (nd2andmasksfound,filterednd2list)

while_counter =1
ctrl_folder_path = ""
ctrl_nd2_list=[]
ctrl_stats_list=[]


while while_counter<2 :#infinite while loop until arm_count.isdigit breaks
    print("enter path of folder with ctrl nd2s and masks")
    time.sleep(0.2)
    ctrl_folder_path = input()
    
    if os.path.exists(ctrl_folder_path):
        os.chdir(ctrl_folder_path)
        for file in glob.glob("*.nd2"):
            ctrl_nd2_list.append(os.path.splitext(file)[0])        

        searchfornd2andmasksout=searchfornd2andmasks(ctrl_folder_path,ctrl_nd2_list)
        filteredctrl_nd2_list=searchfornd2andmasksout[1]
        if searchfornd2andmasksout[0]==1:  
            break
        else:
            print("folder does not have nd2 files and/or associated masks")
    else:
        print("invalid path")


for filenaname in ctrl_nd2_list:
    try:
        ctrl_stats_list.append(autofluorescence(ctrl_folder_path+"\\"+filenaname))
        print("analysis on "+filenaname+" successful")
    except:
        print("problem with "+filenaname+", ignorning")
        continue
        
def calculate_ctrls(ctrl_stats_list):
    nuclearnormlist=[]
    cytoplasmicnormlist=[]
    ctrlbackgroundlist=[]

    for list in ctrl_stats_list:
        nuclearnormlist.append(list[0])
        cytoplasmicnormlist.append(list[1])
        ctrlbackgroundlist.append(list[2])
        
    nuclearnorm=statistics.median(nuclearnormlist)
    cytoplasmicnorm=statistics.median(cytoplasmicnormlist)
    ctrlbackground=statistics.median(ctrlbackgroundlist)
    
    return([nuclearnorm,cytoplasmicnorm,ctrlbackground])


calculatedctrls=calculate_ctrls(ctrl_stats_list)
nuclearnorm=calculatedctrls[0]
cytoplasmicnorm=calculatedctrls[1]
print("nuclear ctrl="+str(nuclearnorm))
print("cytoplasmic ctrl="+str(cytoplasmicnorm))   

def experimentuserinput():
    exp_folder_path=''
    pearsonscolocalizationcondition=0
    while while_counter<2 :#infinite while loop until arm_count.isdigit breaks
        print("------------------------------------------------") 
        print("enter path of folder with experiment nd2 files and masks")
        time.sleep(0.2)
        exp_folder_path = input()
        exp_nd2_list=[]

        if os.path.exists(exp_folder_path):
            os.chdir(exp_folder_path)
            for file in glob.glob("*.nd2"):
                exp_nd2_list.append(os.path.splitext(file)[0])        

            searchfornd2andmasksout=searchfornd2andmasks(exp_folder_path,exp_nd2_list)
            filteredexp_nd2_list=searchfornd2andmasksout[1]
            if searchfornd2andmasksout[0]==1:  
                break

            else:
                print("folder does not have nd2 files and/or associated masks")
        else:
            print("invalid path")


#    while while_counter<2 :#infinite while loop until arm_count.isdigit breaks
#        print("------------------------------------------------") 
#        print("enter 1 if you want to run pearsons colocalization, enter 0 if not")
#        time.sleep(0.2)
#        pearsonscolocalizationcondition = input()

#        if(pearsonscolocalizationcondition=="1" or pearsonscolocalizationcondition=="0"):
#            pearsonscolocalizationcondition = int(pearsonscolocalizationcondition)
#            break

#        else:
#            print("bad input, try again")

    pearsonscolocalizationcondition=1
    return [filteredexp_nd2_list,exp_folder_path,pearsonscolocalizationcondition]

def analysefolder():
    experimentuserinputout=experimentuserinput()
    exp_nd2_list=experimentuserinputout[0]
    exp_folder_path=experimentuserinputout[1]
    pearsonscolocalizationcondition=experimentuserinputout[2]


    exp_stats_list=[["file name","average nuclear/cytoplasmic intensity ratio","median  nuclear/cytoplasmic intensity ratio","proportion cells with intensity ratio>2","average total signal","average Pearson's colocalization coefficient","median background signal"]]

    for filenaname in exp_nd2_list:
        try:   
            temporaryanalyzeimage=analyzeimage(exp_folder_path+"\\"+filenaname,cytoplasmicnorm,nuclearnorm,pearsonscolocalizationcondition)
            
            avgpearsons="n/a"
            if len(temporaryanalyzeimage[0][2])>0:
                avgpearsons=Avg(temporaryanalyzeimage[0][2])
            

            exp_stats_list.append([filenaname,Avg(temporaryanalyzeimage[0][0]),statistics.median(temporaryanalyzeimage[0][0]),len([1 for i in temporaryanalyzeimage[0][0] if i > 2])/len(temporaryanalyzeimage[0][0]),Avg(temporaryanalyzeimage[0][1]),avgpearsons,temporaryanalyzeimage[1]])    #avg.?
            
            np.savetxt(exp_folder_path+"\\"+filenaname+"celllist.csv", temporaryanalyzeimage[0], delimiter=',')#:(

            with open(exp_folder_path+"\\"+filenaname+"celllist.csv", "ab") as f:
                
                f.write(str(temporaryanalyzeimage[1]).encode('utf-8'))
                f.write(b" \n row 1: nuclear signal/cytoplasmic signal | row 2: total average signal | row 3: Pearson's colocalization coefficient | row 4: median background signal")
                
                print("analysis on "+filenaname+" successful")

        except:
            print("problem with "+filenaname+", ignorning")
            continue       

    exp_stats_frame=pd.DataFrame(exp_stats_list)
    print(exp_stats_frame)
    exp_stats_frame.to_csv("!imageaverages.csv")
    
analysefolder()
    
while while_counter<2 :#infinite while loop until arm_count.isdigit breaks
    print("------------------------------------------------") 
    print("enter 1 if you want to analyze another folder with the same controls, enter 0 if not")
    time.sleep(0.2)
    continueinput = input()
    
    if(continueinput=="1" or continueinput=="0"):
        if continueinput=="0":
            print("image analysis complete; data in csv files in experimental folder")
            break
            
        if continueinput=="1":
            analysefolder()

    else:
        print("bad input, try again")


enter path of folder with ctrl nd2s and masks
D:\Users\Titan\Downloads\a days work 12_15_23\1516 ctrl - Copy
analysis on 1516ctrl successful
analysis on 1516ctrl001 successful
analysis on 1516ctrl002 successful
analysis on 1516ctrl003 successful
analysis on 1516ctrl004 successful
analysis on 1516ctrl005 successful
analysis on 1516ctrl006 successful
nuclear ctrl=795.0011882648602
cytoplasmic ctrl=547.5649944881
------------------------------------------------
enter path of folder with experiment nd2 files and masks
D:\Users\Titan\Downloads\a days work 12_15_23\3610 experiment
analysis on 3610.0min successful
analysis on 3610.0min003 successful
analysis on 3610.120min.AA.030 successful
analysis on 3610.120min.AA.031 successful
analysis on 3610.120min.Gln.038 successful
analysis on 3610.120min.Gln.039 successful
analysis on 3610.120min.Pro.032 successful
analysis on 3610.120min.Pro.033 successful
analysis on 3610.15min.AA.005 successful
analysis on 3610.15min.Pro.004 successful
analysis o